In [1]:
import sys
import threading
from math import gcd

def main():
    import sys
    input = sys.stdin.read
    data = input().split()
    
    T = int(data[0])
    queries = [(int(data[i]), int(data[i+1])) for i in range(1, 2*T+1, 2)]
    
    N = 10**6 + 1
    # Precompute Euler's Totient function (phi)
    phi = list(range(N))
    for i in range(2, N):
        if phi[i] == i:
            for j in range(i, N, i):
                phi[j] -= phi[j] // i

    # Precompute prefix sums of phi
    prefix = [0] * N
    for i in range(1, N):
        prefix[i] = prefix[i-1] + phi[i]

    # Function to count valid pairs in range [L, R]
    def count_valid_pairs(L, R):
        res = 0
        for p in range(1, R // 2 + 1):
            low = max((L + p - 1) // p, 1)
            high = R // p
            if high > low:
                res += prefix[high] - prefix[low]
        return res

    for L, R in queries:
        print(count_valid_pairs(L, R))

threading.Thread(target=main).start()

Exception in thread Thread-3 (main):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\olang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\olang\AppData\Local\Temp\ipykernel_5908\3219835340.py", line 10, in main
    T = int(data[0])
            ~~~~^^^
IndexError: list index out of range


In [2]:

# PyPy3 solution

import sys
import math

input = sys.stdin.readline

MAX_N = 10**6

# Step 1: Sieve of Eratosthenes to find all primes up to MAX_N
is_prime = [True] * (MAX_N + 1)
is_prime[0] = is_prime[1] = False
for i in range(2, int(MAX_N ** 0.5) + 1):
    if is_prime[i]:
        for j in range(i * i, MAX_N + 1, i):
            is_prime[j] = False

primes = [i for i, p in enumerate(is_prime) if p]

# Step 2: For each number, find valid p such that n = p * q where p and q are primes
count = [0] * (MAX_N + 1)

for p in primes:
    for q in primes:
        val = p * q
        if val > MAX_N:
            break
        count[val] += 1

# Step 3: Prefix sum for fast range queries
prefix = [0] * (MAX_N + 1)
for i in range(1, MAX_N + 1):
    prefix[i] = prefix[i - 1] + count[i]

# Step 4: Answer queries
T = int(input())
for _ in range(T):
    L, R = map(int, input().split())
    total = prefix[R] - prefix[L - 1]
    print(total)


Output of this code

SyntaxError: invalid syntax (1233185082.py, line 43)

In [3]:
import sys
# Set recursion limit higher for potential deep calls (though not expected here)
# sys.setrecursionlimit(2000000)

# Maximum possible cake size
MAX_N = 1000000

def solve():
    """
    Precomputes the total number of valid pairs (C_i, C_j) in the range [1, R] 
    for all R up to MAX_N, and then answers T range queries in O(1) time each.
    
    A pair (C_i, C_j) is valid if there exists a prime p such that C_i = p*q_i 
    and C_j = p*q_j, where q_i and q_j are also prime.
    """
    
    # 1. Sieve of Eratosthenes to find all primes up to MAX_N
    is_prime = [True] * (MAX_N + 1)
    is_prime[0] = is_prime[1] = False
    primes = []
    
    for p in range(2, MAX_N + 1):
        if is_prime[p]:
            primes.append(p)
            for i in range(p * p, MAX_N + 1, p):
                is_prime[i] = False
    
    # Primes list for efficient iteration over prime slice sizes p.
    
    # 2. Precompute PrimeSliceFactors[C]: 
    # List of primes p such that C = p * q, where q is also prime.
    # This set P(C) contains the prime slice sizes p that make C valid.
    prime_slice_factors = [[] for _ in range(MAX_N + 1)]
    
    # Iterate through all combinations of two primes p1 and p2 (p1 <= p2)
    for p1 in primes:
        # Optimization: C = p1 * p2 must be <= MAX_N
        if p1 * p1 > MAX_N:
            break
            
        for p2 in primes:
            if p2 < p1:
                continue
            
            C = p1 * p2
            if C > MAX_N:
                # Since p2 is increasing, we can break the inner loop
                break
            
            # Add p1 as a valid slice factor for C
            prime_slice_factors[C].append(p1)
            
            # If p1 != p2, then p2 is also a valid slice factor (C/p2 = p1 is prime)
            if p1 != p2:
                prime_slice_factors[C].append(p2)
                
    # 3. Precompute TotalPairs[R] (Prefix Sum Array)
    # TotalPairs[R] = sum of all valid pairs (C_i, C_j) where 1 <= C_i < C_j <= R.
    total_pairs = [0] * (MAX_N + 1)
    
    # ActiveCounts[p] stores the count of cakes C < R that are sliceable by prime p.
    # This acts as C_p[R-1] in the derivation.
    # We only need to track counts for primes up to MAX_N / 2 (since p * q <= 10^6 and q >= 2).
    # Since we are iterating on C, we only need to track the count of cakes *before* C.
    max_active_prime = MAX_N // 2 
    
    # Using a dictionary for sparse storage of ActiveCounts is often faster in Python/PyPy 
    # than a large list if many primes are unused, but here we can just use an array 
    # indexed by prime, or simply use the max prime index. Let's use an array indexed
    # by prime number, size up to 500000 (MAX_N/2)
    
    # Optimization: Use an array and map primes to indices for dense storage/speed, 
    # but for simplicity and readability, we'll use a dictionary.
    active_counts = {} # Key: prime p, Value: count of previous valid cakes C < R sliceable by p.

    # R is the current cake size being considered
    for R in range(1, MAX_N + 1):
        # 3.1. Inherit the count from the previous step
        total_pairs[R] = total_pairs[R - 1]
        
        # 3.2. Calculate NewPairs: the total number of valid pairs formed by (R, C_j) 
        # where C_j < R. This is sum(C_p[R-1] for p in P(R)).
        new_pairs = 0
        
        # R is a semiprime if prime_slice_factors[R] is not empty
        for p in prime_slice_factors[R]:
            # R and C_j form a valid pair if C_j is also sliceable by p.
            # ActiveCounts[p] stores the count of such C_j < R.
            new_pairs += active_counts.get(p, 0)
        
        # 3.3. Update the total pair count
        total_pairs[R] += new_pairs
        
        # 3.4. Update ActiveCounts for the next iteration (R+1)
        # R itself is now part of the count for the primes in P(R).
        for p in prime_slice_factors[R]:
            active_counts[p] = active_counts.get(p, 0) + 1


    # 4. Query Handling (O(1) per query)
    try:
        # Read the number of puzzles T
        T = int(sys.stdin.readline())
    except:
        return

    output = []
    for _ in range(T):
        try:
            line = sys.stdin.readline().split()
            L = int(line[0])
            R = int(line[1])
        except:
            continue
            
        if L > R or R > MAX_N:
             # Based on constraints, this shouldn't happen, but good practice to check.
            answer = 0
        else:
            # The count of valid pairs in [L, R] is:
            # (Total pairs in [1, R]) - (Total pairs in [1, L-1])
            answer = total_pairs[R] - total_pairs[L - 1]
            
        output.append(str(answer))

    # Print all results separated by a newline
    sys.stdout.write("\n".join(output) + "\n")

if __name__ == "__main__":
    # Optimize input/output for PyPy3 environment
    if 'pypy' in sys.version.lower():
        # PyPy is usually faster with simple list appends and printing at the end.
        pass
    else:
        # For standard CPython, this may slightly speed up I/O.
        # It's omitted here since PyPy is requested and is generally fast enough.
        pass
        
    solve()
